The purpose of this notebook is to teach students how to run and interpret metagenome read recruitment onto SAGs.

## Data Environment Setup

We will be downloading (or copying over) metagenome data from NCBI to use for this project.

I've done this in advance of the lesson, but I want you all to see how it is done for future projects.

First we'll create a project directory for this lesson.  
Then, we'll create a data directory in storage.
Finally, we'll link our data directory to our project directory.  The code will look something like this:

I have made a directory for metagenomic data within our storage directory:  
```
$ mkdir -p /mnt/storage/data/day3AM_metagenomes/
```

Make a project directory:
```
$ mkdir ~/storage/user_lab/{your_username}/metagenome_recruitment_lesson
```

Now navigate into your project directory ( ~/storage/user_lab/{your_username}/metagenome_recruitment_lesson) and start a jupyter notebook using the file system navigator side bar.  

In [ ]:
# make a directory for data within your storage directory:
!mkdir -p ~/storage/user_lab/julia/metagenome_recruitment/data/

## Software Environment Setup

The software needed for this tutorial can be found in the conda env config file ./coverm_env.yml

It is pre-installed on the jupyter hub.  To activate it in the terminal type:

```
conda activate coverM
```

**TODO: this is currently a local environment for me... I need to install it as root so that everyone has access.**

To use it in a jupyter notebook, select the 'coverm' kernel on the upper right-hand side of the screen.

## Data Setup

For this lesson, we are going to use the SAGs that we have been working with all week, AG-910, to examine their abundance over time at BATS using metagenomic read recruitment.  

Here's the data we will access:

We will download metagenomic reads from BATS from this bioproject: https://www.ncbi.nlm.nih.gov/bioproject?term=PRJNA385855

I've downloaded a metadata sheet with all sra metagenomes from this bioproject to: ./data/PRJNA385855_sra_metadata.csv

In [24]:
import pandas as pd

df = pd.read_csv("./data/PRJNA385855_sra_metadata.csv", sep = ",")

df.columns

Index(['Run', 'Assay Type', 'AvgSpotLen', 'Bases', 'BioProject', 'BioSample',
       'BioSampleModel', 'bottle_id', 'Bytes', 'Center Name',
       'Collection_date', 'Consent', 'cruise_id', 'DATASTORE filetype',
       'DATASTORE provider', 'DATASTORE region', 'Depth', 'env_biome',
       'env_feature', 'env_material', 'Experiment', 'geo_loc_name_country',
       'geo_loc_name_country_continent', 'geo_loc_name', 'Instrument',
       'isolation-source', 'lat_lon', 'Library Name', 'LibraryLayout',
       'LibrarySelection', 'LibrarySource', 'Organism', 'Platform',
       'ReleaseDate', 'Sample Name', 'SRA Study'],
      dtype='object')

Let's look at the metagenomes within this dataframe:

In [25]:
df[['Run','Assay Type','Collection_date','cruise_id','Depth']].head(20)

,Run,Assay Type,Collection_date,cruise_id,Depth
0,SRR6507277,WGS,2009-08-19,HOT214,5m
1,SRR6507278,WGS,2009-11-04,HOT216,100m
2,SRR6507279,WGS,2009-07-14,BATS248,10m
3,SRR6507280,WGS,2009-11-07,BATS252,100m
4,SRR5720219,WGS,2004-10-10,HOT164,5m
5,SRR5720221,WGS,2004-08-15,HOT162,175m
6,SRR5720223,WGS,2004-09-28,HOT163,115m
7,SRR5720224,WGS,2004-09-28,HOT163,175m
8,SRR5720226,WGS,2004-06-15,HOT160,175m
9,SRR5720228,WGS,2004-08-15,HOT162,100m


Our SAG plate is from BATS248.  We are interested in examining the abundance of SAGs in the surface at BATS within this metagenome collection, so let's find just those metagenomes collected at BATS from either 1m or 10m depth.

In [30]:
# dataframe of metagenomes of interest
mgoi = df[df['cruise_id'].str.contains('BATS') & df['Depth'].isin(['10m','1m'])][['Run','Collection_date','cruise_id','BioSample','Depth']].sort_values(by = 'Collection_date')

# going to save this table to file
mgoi.to_csv("./data/bats_metagenomes_of_interest.csv", index=False)

print("There are", len(mgoi), "metagenomes that we are interested in downloading")
mgoi

There are 21 metagenomes that we are interested in downloading


,Run,Collection_date,cruise_id,BioSample,Depth
74,SRR5720233,2003-02-21,BATS173,SAMN07137079,1m
14,SRR5720238,2003-03-22,BATS174,SAMN07137082,1m
119,SRR5720327,2003-04-22,BATS175,SAMN07137064,10m
99,SRR5720283,2003-05-20,BATS176,SAMN07137103,1m
75,SRR5720235,2003-07-15,BATS178,SAMN07137085,10m
38,SRR5720286,2003-08-12,BATS179,SAMN07137088,10m
64,SRR5720332,2003-10-07,BATS181,SAMN07137067,1m
96,SRR5720276,2003-11-04,BATS182,SAMN07137106,1m
90,SRR5720262,2003-12-02,BATS183,SAMN07137109,1m
124,SRR5720338,2004-01-27,BATS184,SAMN07137070,1m


Just for ease of access, I'm going to print out another small text file that has just the runids of the metagenomes we are interested in downloading into the directory we will download the files into:

In [29]:
with open('/home/jupyter-julia/storage/user_lab/julia/metagenome_recruitment/data/metagenomes_to_download.txt', 'w') as oh:
    for run in mgoi['Run']:
        print(run, file = oh)

I'm next going to download these metagenomes from ncbi using the fastq-dump function from the sra-tools package.  Some parameters I'll use, I'm going to skip technical reads, and download only 1000000 reads from each metagenome.

Open up a temrinal window and then run:

```
conda activate coverm

cd ~/storage/user_lab/julia/metagenome_recruitment/data/

while read p; do
  fastq-dump --split-files --skip-technical -N 0 -X 1000000 --gzip --readids "$p"
done <metagenomes_to_download.txt
```

Let's check which metagenomes were downloaded, using the python package 'glob' which identifies files using wildcards, and returns them as a list.

In [31]:
from glob import glob

mgs = glob('/home/jupyter-julia/storage/user_lab/julia/metagenome_recruitment/data/*_1.fastq.gz')

Now I'm going to use what's called a 'list comprehension' to extract the runids from this list of files:

In [35]:
runids = [i.split("/")[-1].split("_")[0] for i in mgs]
len(runids)

19

Hmmmm... two SRA archives are missing, I wonder which ones those are.

In [39]:
mgoi.loc[~mgoi['Run'].isin(runids),'downloaded'] = 'no'
mgoi['downloaded'] = mgoi['downloaded'].fillna('yes')
mgoi

,Run,Collection_date,cruise_id,BioSample,Depth,downloaded
74,SRR5720233,2003-02-21,BATS173,SAMN07137079,1m,yes
14,SRR5720238,2003-03-22,BATS174,SAMN07137082,1m,yes
119,SRR5720327,2003-04-22,BATS175,SAMN07137064,10m,yes
99,SRR5720283,2003-05-20,BATS176,SAMN07137103,1m,yes
75,SRR5720235,2003-07-15,BATS178,SAMN07137085,10m,yes
38,SRR5720286,2003-08-12,BATS179,SAMN07137088,10m,yes
64,SRR5720332,2003-10-07,BATS181,SAMN07137067,1m,yes
96,SRR5720276,2003-11-04,BATS182,SAMN07137106,1m,no
90,SRR5720262,2003-12-02,BATS183,SAMN07137109,1m,yes
124,SRR5720338,2004-01-27,BATS184,SAMN07137070,1m,yes


In [41]:
mgoi.to_csv("./data/bats_metagenomes_of_interest.csv", index=False)

The error I see in the terminal says:

```
2022-03-23T19:51:04 fastq-dump.2.8.0 err: no error - error with http open 'https://sra-downloadb.be-md.ncbi.nlm.nih.gov/sos2/sra-pub-run-11/SRR5720276/SRR5720276.1'
2022-03-23T19:51:04 fastq-dump.2.8.0 err: item not found while constructing within virtual database module - the path 'SRR5720276' cannot be opened as database or table
```

For whatever reason, these two metagenomes can't be downloaded. I'm OK with that, we'll have enough information with the metagenomes we recovered to look into SAG abundance at BATS over time.